In [1]:
import torch
from torch import nn
from torch import optim
from torch_geometric.data import NeighborSampler

%matplotlib inline
import matplotlib.pyplot as plt

from modeling import HybridNetwork, train
from dataset import HybridDataset

## Dataset

In [2]:
dataset = HybridDataset("../graphs/restaurants_test1.gpickle",
                        "../graphs/2017-2018_user_network.gpickle",
                        "../datasets/2017-2018_visited_users.csv",
                        split=(0.8, 0.1, 0.1))

Number of restaurants: 29963
Number of neighbors: 491371
converting restaurant graph to pyg graph... done!
Number of users: 579604
Number of friends: 1560849
converting restaurant graph to pyg graph... done!


In [3]:
train_loader = NeighborSampler(dataset.res_pyg_graph.edge_index, node_idx=dataset.train_index,
                               sizes=[-1, -1], batch_size=dataset.train_index.shape[0], shuffle=True)
all_loader = NeighborSampler(dataset.res_pyg_graph.edge_index, node_idx=None, sizes=[-1, -1],
                                  batch_size=dataset.res_pyg_graph.edge_index.shape[1], shuffle=False)

In [4]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

dataset.to(device)  # this does not return anything
model = HybridNetwork(dataset.num_res_features,
                      dataset.num_user_features,
                      64,
                      dataset.num_class,
                      num_layers=2,
                      model="GraphSage",
                      k=5).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
loss = nn.CrossEntropyLoss()

In [5]:
train(model, 150, dataset, train_loader, optimizer, loss, device, k=5, all_loader=all_loader)

Epoch 01, Loss: 1.1005, Approx. Train: 0.3365
Train: 0.3309, Val: 0.3268, Test: 0.3270
Epoch 02, Loss: 1.1466, Approx. Train: 0.3309
Train: 0.4101, Val: 0.3919, Test: 0.3867
Epoch 03, Loss: 1.0919, Approx. Train: 0.3969
Train: 0.4180, Val: 0.4182, Test: 0.4154
Epoch 04, Loss: 1.0871, Approx. Train: 0.3860
Train: 0.3769, Val: 0.3782, Test: 0.3714
Epoch 05, Loss: 1.0777, Approx. Train: 0.3806
Train: 0.4352, Val: 0.4252, Test: 0.4221
Epoch 06, Loss: 1.0586, Approx. Train: 0.4239
Train: 0.4985, Val: 0.4830, Test: 0.4831
Epoch 07, Loss: 1.0433, Approx. Train: 0.4679
Train: 0.4937, Val: 0.4760, Test: 0.4801
Epoch 08, Loss: 1.0277, Approx. Train: 0.4748
Train: 0.4889, Val: 0.4700, Test: 0.4758
Epoch 09, Loss: 1.0144, Approx. Train: 0.4756
Train: 0.5088, Val: 0.4873, Test: 0.5035
Epoch 10, Loss: 1.0002, Approx. Train: 0.4895
Train: 0.5141, Val: 0.4977, Test: 0.5015
Epoch 11, Loss: 0.9923, Approx. Train: 0.4982
Train: 0.5102, Val: 0.4967, Test: 0.4962
Epoch 12, Loss: 0.9856, Approx. Train: 0.49